In [1]:
# TODO: Load audio files, Preprocess as melspec, save as np file
PATH = '../datasets/gtzan/Data/genres_original'

In [9]:
import preprocess

In [ ]:
preprocess.process_files(PATH, 3, 1)

In [6]:
df = pd.DataFrame(np.load(f'./working_data/3s_block_1s_hop/data.npy',allow_pickle=True),columns=['audio','melspec','label'])
X = df['melspec'].to_numpy()
y = df['label'].to_numpy()

,audio,melspec,label
0,"[0.7092849612236023, 0.5320729613304138, 0.286...","[[-10.983954171604147, -7.614372770525382, -6....",pop
1,"[0.06586777418851852, 0.08585505932569504, 0.0...","[[-34.545366849410655, -31.90809386292815, -32...",pop
2,"[-0.15317505598068237, -0.1191829964518547, -0...","[[-55.4434162179103, -62.055048445114515, -69....",pop
3,"[0.2638251781463623, 0.29334208369255066, 0.30...","[[-16.821077549174735, -9.7552262265737, -8.80...",pop
4,"[-0.21769361197948456, -0.01245081052184105, -...","[[-7.420879304207965, -4.61488802560244, -5.54...",pop
...,...,...,...
30954,"[0.06790105998516083, 0.06367199122905731, 0.0...","[[-23.257755185198793, -23.440252443017314, -2...",reggae
30955,"[-0.011190357618033886, -0.004760471172630787,...","[[-38.93165692179263, -36.38457109964692, -33....",reggae
30956,"[-0.23096606135368347, -0.23396214842796326, -...","[[-9.466772240359223, -10.443182129367155, -13...",reggae
30957,"[0.056767627596855164, 0.05592524632811546, 0....","[[-30.40183162320119, -29.573339673666407, -31...",reggae


In [12]:
X = df['melspec'].to_numpy()
y = df['label'].to_numpy()

In [27]:
import melspec

SyntaxError: invalid syntax (1656210301.py, line 1)

: 

# TAU Scraping

In [2]:
import os
PATH = '../datasets/TAU/audio' # Relative from 'Vedant' folder

In [8]:
import pandas as pd
df=pd.read_csv('../datasets/TAU/metadata/meta.csv', sep='\t')

In [9]:
df

,filename,scene_label,identifier,source_label
0,audio/airport-lisbon-1000-40000-a.wav,airport,lisbon-1000,a
1,audio/bus-lyon-1001-40001-a.wav,bus,lyon-1001,a
2,audio/shopping_mall-lisbon-1002-40002-a.wav,shopping_mall,lisbon-1002,a
3,audio/street_pedestrian-lyon-1003-40003-a.wav,street_pedestrian,lyon-1003,a
4,audio/street_pedestrian-lisbon-1004-40004-a.wav,street_pedestrian,lisbon-1004,a
...,...,...,...,...
14395,audio/tram-vienna-285-8635-a.wav,tram,vienna-285,a
14396,audio/tram-vienna-285-8636-a.wav,tram,vienna-285,a
14397,audio/tram-vienna-285-8637-a.wav,tram,vienna-285,a
14398,audio/tram-vienna-285-8638-a.wav,tram,vienna-285,a


In [10]:
df.groupby('scene_label').count()

,filename,identifier,source_label
scene_label,,,
airport,1440,1440,1440
bus,1440,1440,1440
metro,1440,1440,1440
metro_station,1440,1440,1440
park,1440,1440,1440
public_square,1440,1440,1440
shopping_mall,1440,1440,1440
street_pedestrian,1440,1440,1440
street_traffic,1440,1440,1440


In [12]:
import librosa
x,sr = librosa.load('../datasets/TAU/audio/airport-lisbon-1000-40000-a.wav')

/usr/local/lib/python3.8/dist-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: '../datasets/TAU/audio/airport-lisbon-1000-40000-a.wav'

In [4]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
import glob
import os
import random
from audiomentations import Compose, AddGaussianNoise, AddBackgroundNoise, ApplyImpulseResponse


In [1]:
a = 0.5   # blend factor
min_snr = 1.0
max_snr = 10
aug_type = 'both'
n_BG = 10
n_IR = 5
audioPath = f'../datasets/gtzan/Data/genres_original'
irPath = f'../datasets/IR_MIT/'
bgPath = '../datasets/TAU/audio/'



In [15]:
random.seed(42)
def readNoisePaths(bgPath, n):
    # Read n BGs in an array
    allBGPaths = glob.glob(bgPath + "*.wav")
    # Select n BG files randomly
    nBGPaths = random.sample(allBGPaths, n)
    return nBGPaths
df = pd.DataFrame(columns = ['audioPath', 'IRPath', 'BGPath', 'fname', 'label'])
dirName, subdirList, _ = next(os.walk(audioPath))
i = 0
for subdir in subdirList:
    _, _, fileList = next(os.walk(os.path.join(dirName,subdir)))
    # print(fileList)
    for filename in tqdm(fileList):
        path = os.path.join(dirName, subdir, filename)
        fname = filename.split('/')[-1][:-4]
        nBGPaths = readNoisePaths(bgPath, n_BG)
        for noisePath in nBGPaths:
            nIRPaths = readNoisePaths(irPath, n_IR)
            for IRnoisePath in nIRPaths:
                df = df.append({'audioPath' : path, 'IRPath' : IRnoisePath, 'BGPath' : noisePath, 'fname' : fname, 'label' : subdir}, ignore_index = True)

 27%|██▋       | 27/100 [00:05<00:14,  5.09it/s]

In [14]:
df1

,audioPath,IRPath,BGPath,fname,label
0,../datasets/gtzan/Data/genres_original/pop/pop...,../datasets/IR_MIT/h037_Classroom_5txts.wav,../datasets/TAU/audio/metro-stockholm-227-6850...,pop.00001,pop
1,../datasets/gtzan/Data/genres_original/pop/pop...,../datasets/IR_MIT/h266_MITCampus_StduentLoung...,../datasets/TAU/audio/metro-stockholm-227-6850...,pop.00001,pop
2,../datasets/gtzan/Data/genres_original/pop/pop...,../datasets/IR_MIT/h253_Classroom_1txts.wav,../datasets/TAU/audio/metro-stockholm-227-6850...,pop.00001,pop
3,../datasets/gtzan/Data/genres_original/pop/pop...,../datasets/IR_MIT/h247_Classroom_1txts.wav,../datasets/TAU/audio/metro-stockholm-227-6850...,pop.00001,pop
4,../datasets/gtzan/Data/genres_original/pop/pop...,../datasets/IR_MIT/h023_Bedroom_9txts.wav,../datasets/TAU/audio/metro-stockholm-227-6850...,pop.00001,pop
...,...,...,...,...,...
49995,../datasets/gtzan/Data/genres_original/reggae/...,../datasets/IR_MIT/h003_Office_LargeBrickWalle...,../datasets/TAU/audio/public_square-stockholm-...,reggae.00053,reggae
49996,../datasets/gtzan/Data/genres_original/reggae/...,../datasets/IR_MIT/h002_Bedroom_62txts.wav,../datasets/TAU/audio/public_square-stockholm-...,reggae.00053,reggae
49997,../datasets/gtzan/Data/genres_original/reggae/...,../datasets/IR_MIT/h052_Gym_WeightRoom_3txts.wav,../datasets/TAU/audio/public_square-stockholm-...,reggae.00053,reggae
49998,../datasets/gtzan/Data/genres_original/reggae/...,../datasets/IR_MIT/h225_StreetsOfCambridge_1tx...,../datasets/TAU/audio/public_square-stockholm-...,reggae.00053,reggae


In [ ]:
print("Without seed")
print(len(df1['IRPath'].unique()))
print(len(df1['BGPath'].unique()))

print("With seed")
print(len(df['IRPath'].unique()))
print(len(df['BGPath'].unique()))

# PREPROCESSING AND DATALOADER FOR AUGMENTED DATASET

In [46]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib.pyplot as plt
import math
import soundfile as sf
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, mixed_precision
from tensorflow.keras.utils import plot_model
from sklearn import preprocessing
import sys
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
import matplotlib.pyplot as plt
import json
import gc



In [47]:
df = pd.DataFrame(np.load('../datasets/gtzan10sAug/Final/features/a0.5_min1.0_max10.0_3.0s_block_0.5s_hop.npy', allow_pickle=True), columns=['melspec','label','filename'])
X = np.stack(df[['melspec','filename']].values)
y = df['label'].to_numpy()
names = df['filename'].to_numpy()
del df
gc.collect()
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [2]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


In [50]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=42)

In [51]:
x_test, x_val, y_test, y_val = train_test_split(x_val, y_val, test_size=0.50, shuffle=True, random_state=42)

In [53]:
len(y_test) + len(y_train) + len(y_val)

710761

In [5]:
np.save('x_train.npy', x_train)
np.save('x_test.npy',x_test)
np.save('x_val.npy',x_val)
np.save('y_train.npy', y_train)
np.save('y_test.npy',y_test)
np.save('y_val.npy', y_val)


In [ ]:
df=pd.DataFrame()
df['melspec'] = list(x_train[:,0])
df['label'] = y_train
df['filename'] = x_train[:,1]
for idx, row in tqdm(df.iterrows()):
    # filename = row['filename']
    np.save(f'../datasets/gtzan10sAug/vedant/train/{idx}.npy',row[['melspec','label']])

In [65]:
df=pd.DataFrame()
df['melspec'] = list(x_test[:,0])
df['label'] = y_test
df['filename'] = x_test[:,1]
for idx, row in tqdm(df.iterrows()):
    # filename = row['filename']
    np.save(f'../datasets/gtzan10sAug/vedant/test/{idx}.npy',row[['melspec','label']])

106614it [00:53, 2001.23it/s]


In [66]:
df=pd.DataFrame()
df['melspec'] = list(x_val[:,0])
df['label'] = y_val
df['filename'] = x_val[:,1]
for idx, row in tqdm(df.iterrows()):
    # filename = row['filename']
    np.save(f'../datasets/gtzan10sAug/vedant/val/{idx}.npy',row[['melspec','label']])

106615it [00:53, 1988.93it/s]


In [12]:
row[['melspec','label']]

melspec    [[-23.601372, -22.739994, -20.685085, -17.7462...
label                                                    pop
Name: 0, dtype: object

In [13]:
np.save('test.npy',row[['melspec','label']])

In [19]:
import os
_,_,filenames = next(os.walk('../datasets/gtzan10sAug/vedant/train'))

In [20]:
len(filenames)

44955

In [24]:
np.load(os.path.join('../datasets/gtzan10sAug/vedant/train','blues.00000.0_metro-lyon-1064-40512-a_h082_HomeFoyer_2txts.wav.npy'), allow_pickle=True)[0]

array([[-17.752384, -16.114372, -18.592854, ..., -17.36829 , -16.564209,
        -17.912159],
       [-18.907047, -15.063114, -15.949604, ..., -21.083767, -13.481085,
        -11.707794],
       [-22.559961, -17.943682, -17.578224, ..., -20.885958, -15.600981,
        -14.335256],
       ...,
       [-71.23194 , -72.94471 , -75.63745 , ..., -79.13965 , -70.388054,
        -62.018204],
       [-72.10234 , -75.0163  , -78.81941 , ..., -80.      , -70.87243 ,
        -62.156807],
       [-73.01843 , -78.82287 , -80.      , ..., -80.      , -71.16549 ,
        -62.198265]], dtype=float32)

In [25]:
a = np.zeros((32,128,130))
a[0] = np.load(os.path.join('../datasets/gtzan10sAug/vedant/train','blues.00000.0_metro-lyon-1064-40512-a_h082_HomeFoyer_2txts.wav.npy'), allow_pickle=True)[0]
a

array([[[-17.75238419, -16.11437225, -18.59285355, ..., -17.36828995,
         -16.56420898, -17.91215897],
        [-18.90704727, -15.06311417, -15.94960403, ..., -21.08376694,
         -13.48108482, -11.70779419],
        [-22.55996132, -17.94368172, -17.57822418, ..., -20.88595772,
         -15.60098076, -14.33525562],
        ...,
        [-71.23194122, -72.94470978, -75.63745117, ..., -79.13964844,
         -70.38805389, -62.01820374],
        [-72.1023407 , -75.01629639, -78.81941223, ..., -80.        ,
         -70.87242889, -62.15680695],
        [-73.01843262, -78.82286835, -80.        , ..., -80.        ,
         -71.1654892 , -62.19826508]],

       [[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        ...,


In [67]:
class AugmentedDataset(keras.utils.Sequence):
    def __init__(self, mode, batch_size):
        self.batch_size = batch_size
        self.mode = mode
        print(self.mode)
        if self.mode == 'train':
            self.data_path = '../datasets/gtzan10sAug/vedant/train'
        if self.mode == 'test':
            self.data_path = '../datasets/gtzan10sAug/vedant/test'
        if self.mode == 'val':
            self.data_path = '../datasets/gtzan10sAug/vedant/val'
        _,_,self.filenames = next(os.walk(self.data_path))
        print(len(self.filenames))

    def __len__(self):
        return int(len(self.filenames) // self.batch_size)

    def __getitem__(self,idx):
        X = np.empty((self.batch_size, 128,130))
        y = np.empty(self.batch_size)
        batch = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        for i, ID in enumerate(batch):
            tmp = np.load(os.path.join(self.data_path, ID), allow_pickle=True)
            X[i] = tmp[0]
            y[i] = tmp[1]

        # bat = np.array([np.load(os.path.join(self.data_path, x), allow_pickle=True) for x in batch])
        return X,y #bat[:,0], bat[:,1]

In [39]:
_,_,a = next(iter(os.walk('../datasets/gtzan10sAug/vedant/train')))
len(a)

44955

In [68]:
batch_size = 32
train_dataset = AugmentedDataset('train', batch_size)
test_dataset = AugmentedDataset('test', batch_size)
val_dataset = AugmentedDataset('val', batch_size)

train
497532
test
106614
val
106615


In [69]:
32*(len(train_dataset)+ len(val_dataset) + len(test_dataset))

710688

In [45]:
a[0].shape

(128, 130)

In [43]:
a,b = next(iter(train_dataset))
print(a)
print(b)

[array([[-34.500694, -33.242523, -30.692486, ..., -22.4065  , -26.499474,
         -28.723408],
        [-33.037247, -33.36597 , -30.984373, ..., -19.223448, -23.66777 ,
         -27.489506],
        [-36.650776, -32.63643 , -32.53936 , ..., -29.785194, -25.498083,
         -26.271564],
        ...,
        [-73.12674 , -71.38355 , -69.532265, ..., -62.48752 , -64.315125,
         -67.44705 ],
        [-74.42844 , -72.949486, -70.620476, ..., -68.44469 , -70.12893 ,
         -73.836136],
        [-75.131004, -80.      , -80.      , ..., -80.      , -80.      ,
         -80.      ]], dtype=float32)
 array([[ -8.242342 ,  -5.6894674,  -8.421957 , ...,  -5.10923  ,
          -4.6165915,  -4.675352 ],
        [ -9.052369 ,  -9.100551 ,  -9.461972 , ..., -13.622889 ,
         -11.160433 ,  -8.27943  ],
        [-15.741032 , -17.629871 , -16.213617 , ..., -24.652554 ,
         -20.988214 , -18.120441 ],
        ...,
        [-58.7001   , -57.82404  , -59.055614 , ..., -58.450768 ,
         -

In [34]:
len(train_dataset.filenames)

44955

In [32]:
next(iter(train_dataset))

(array([array([[-34.500694, -33.242523, -30.692486, ..., -22.4065  , -26.499474,
                -28.723408],
               [-33.037247, -33.36597 , -30.984373, ..., -19.223448, -23.66777 ,
                -27.489506],
               [-36.650776, -32.63643 , -32.53936 , ..., -29.785194, -25.498083,
                -26.271564],
               ...,
               [-73.12674 , -71.38355 , -69.532265, ..., -62.48752 , -64.315125,
                -67.44705 ],
               [-74.42844 , -72.949486, -70.620476, ..., -68.44469 , -70.12893 ,
                -73.836136],
               [-75.131004, -80.      , -80.      , ..., -80.      , -80.      ,
                -80.      ]], dtype=float32)                                    ,
        array([[ -8.242342 ,  -5.6894674,  -8.421957 , ...,  -5.10923  ,
                 -4.6165915,  -4.675352 ],
               [ -9.052369 ,  -9.100551 ,  -9.461972 , ..., -13.622889 ,
                -11.160433 ,  -8.27943  ],
               [-15.741032 , -17.62

In [36]:
a = next(iter(test_dataset))

In [43]:
len(test_dataset)

41460

In [51]:
df = pd.DataFrame(np.load(f'working_data/{3}s_block_{1}s_hop_{22050}_sr/data.npy',allow_pickle=True),columns=['audio','melspec','label'])
X = np.stack(df['melspec'].values)
y = df['label'].to_numpy()
print(X.shape)
print(y.shape)

(30959, 128, 130)
(30959,)


In [53]:
X[0].shape

(128, 130)

In [44]:
def build_model(name, shape, output):
    if name == 'CNN':
        inputs = layers.Input(shape=shape)
        # layer 1
        x = layers.Reshape((shape[0], shape[1], 1))(inputs)
        x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Dropout(0.1)(x)
        # layer 2
        x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Dropout(0.1)(x)
        # layer 3
        x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Dropout(0.1)(x)
        # last layer
        x = layers.Flatten()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(256, activation='relu')(x)
        outputs = layers.Dense(output, activation='softmax', dtype='float32')(x)
        return keras.Model(inputs=inputs, outputs=outputs)

    elif name == 'ResNet':
        inputs = keras.Input(shape=shape)

        z = layers.Reshape((shape[0], shape[1], 1))(inputs)

        # layer 1
        first = layers.Conv2D(256, 4, padding='same', activation='relu')(z)
        z = layers.Dropout(0.2)(first)
        z = layers.BatchNormalization()(z)
        z = layers.MaxPooling2D(2, 1, padding='same')(z)
        # layer 2
        z = layers.Conv2D(256, 4, padding='same', activation='relu')(z)
        z = layers.Dropout(0.2)(z)
        z = layers.BatchNormalization()(z)
        z = layers.MaxPooling2D(2, 1, padding='same')(z)
        # layer 3
        z = layers.Conv2D(256, 4, padding='same', activation='relu')(z)
        z = layers.Dropout(0.2)(z)
        z = layers.BatchNormalization()(z)
        # Residual block
        z = layers.add([first, z])
        # Parallel pooling
        x = layers.MaxPooling2D(125, 1, padding='same')(z)
        y = layers.AveragePooling2D(125, 1, padding='same')(z)
        x = layers.Flatten()(x)
        y = layers.Flatten()(y)

        z = layers.add([x, y])
        # Dense layers
        z = layers.Dense(300, activation='relu')(z)
        z = layers.Dense(150, activation='relu')(z)
        outputs = layers.Dense(output, activation='softmax', dtype='float32')(z)

        return keras.Model(inputs=inputs, outputs=outputs)
    elif name == 'Xception':
        # EfficientNetB0
        inputs = keras.Input(shape=shape)

        z = layers.Reshape((shape[0], shape[1], 1))(inputs)
        eff = keras.applications.EfficientNetB0(weights=None, input_tensor=z, include_top=False)
        z = eff(z)
        z = layers.GlobalAveragePooling2D()(z)

        outputs = layers.Dense(output, activation='softmax', dtype='float32')(z)

        return keras.Model(inputs=inputs, outputs=outputs)

model = build_model("CNN", (next(iter(train_dataset)).shape[0], next(iter(train_dataset)).shape[1]), 10)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy',   metrics=['acc'])

AttributeError: 'tuple' object has no attribute 'shape'

In [1]:
next(iter(train_dataset))

NameError: name 'train_dataset' is not defined